In [15]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

In [16]:
# Setup OpenAI API key
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [29]:
import json
import requests

def get_current_weather(location, unit="imperial"):
    """Get the current weather in a given location"""
    api_key = os.getenv("OPENWEATHER_API_KEY")

    base_url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&units={unit}&appid={api_key}"
    response = requests.get(base_url)
    data = response.json()

    weather_info = {
        "location": location,
        "temperature": data['main']['temp'],
        "unit": unit,
        "forecast": data['weather'][0]['description'],
    }
    return json.dumps(weather_info)

In [31]:
get_current_weather("Denver")

'{"location": "Denver", "temperature": 33.42, "unit": "imperial", "forecast": "light snow"}'

In [47]:
# Define an OpenAI function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather at a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city, e.g. San Francisco",
                },
                "unit": {"type": "string", "enum": ["imperial", "metric"]},
            },
            "required": ["location"],
        },
    }
]

In [48]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [49]:
client = openai.OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

In [50]:
print(response)

ChatCompletion(id='chatcmpl-8a6QzJMHfW3OYqyXb5xJwJbbfEuVl', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston"\n}', name='get_current_weather'), tool_calls=None))], created=1703615233, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=77, total_tokens=93))


In [51]:
response_msg = response.choices[0].message.function_call
args = json.loads(response_msg.arguments)

get_current_weather(location=args["location"])

'{"location": "Boston", "temperature": 48.92, "unit": "imperial", "forecast": "broken clouds"}'